## Experiments 

In [3]:
import sys 
sys.path.append('/home/mahakaran/NK-paper-12-5-24-version/nk-ml-paper2-2024/pscapes') #add pscapes 

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from pscapes.landscape_models import make_NK
from pscapes.landscape_class import Protein_Landscape
from pscapes.utils import dict_to_np_array

import scipy.sparse as sp

from to

In [5]:
sys.version

'3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]'

In [2]:
AA_ALPHABET = 'ACDEFG'
SEQ_LENGTH  = 6

In [3]:
test_NK_1_dict = make_NK(SEQ_LENGTH, 1, AA_ALPHABET, np.random.uniform)

In [4]:
test_NK_1_np = dict_to_np_array(test_NK_1_dict)

In [5]:
test_NK_1_PL = Protein_Landscape(test_NK_1_np, amino_acids=AA_ALPHABET)

In [6]:
test_NK_1_PL.fit_graph_builder()

Building Protein Graph for entire dataset


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 46656/46656 [00:09<00:00, 4970.87it/s]



            Protein_Landscape(sequences={})
            

In [6]:
test_NK_1_PL.fit_adjacency()

Must compute graph before building adjacency. Computing graph...
Building Protein Graph for entire dataset


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 46656/46656 [00:09<00:00, 4976.75it/s]


Graph built. Computing adjancency...



            Protein_Landscape(sequences={})
            

In [ ]:
test_NK_1_PL.

In [46]:
H = sp.lil_matrix((10,10))

In [49]:
H[[1,3,9],0]=1

In [50]:
H.toarray()

array([[0., 1., 0., 1., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [51]:
def dict_graph_to_adjacency(dict_graph):
    dim = len(dict_graph)
    A   = sp.lil_matrix((dim, dim))
    values = list(dict_graph.values())
    
    for ind, i in enumerate(values): 
        A[ind, i]=1
    return A
    
    
    
    

In [52]:
adj = dict_graph_to_adjacency(test_NK_1_PL.graph)

In [55]:
adj_np = adj[:100,:100].toarray()

In [65]:
f = np.array(list(test_NK_1_PL.graph.values()))

In [69]:
query = f[0][f[0]<100]

In [84]:
query.dtype


dtype('int64')

In [90]:
adj_np[0,query]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])